In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#グラフの描画
%matplotlib inline

In [2]:
files = glob.glob("data/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

/var/folders/2r/1x2sq2h12nq19x6804nzmc_h0000gp/T/ipykernel_18002/1756106423.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/var/folders/2r/1x2sq2h12nq19x6804nzmc_h0000gp/T/ipykernel_18002/1756106423.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [3]:
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
                nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis = 1)

    dis = {
        "1H?1H30":75,
        "30分?60分":45,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期":".25",
        "年第2四半期":".50",
        "年第3四半期":".75",
        "年第4四半期":".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k,j in year.items():
            if k in i:
                year_rep = i.replace(k,j)
        year_list[i] = year_rep
    year_list
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    # lightgbmのためにカテゴリ質的変数ですよと明示する
    for col in ["都道府県名","地区名","最寄駅：名称","間取り","建物の構造","用途","今後の利用目的","都市計画","改装","取引の事情等"]:
        # lgb.train()時のエラーの解消
        # ValueError: pandas dtypes must be int, float or bool.
        df[col] = df[col].astype("category")
    return df

In [4]:
df = data_pre(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 637351 entries, 40108844 to 12009989
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       637351 non-null  int64   
 1   都道府県名         637351 non-null  category
 2   地区名           637060 non-null  category
 3   最寄駅：名称        634732 non-null  category
 4   最寄駅：距離（分）     614306 non-null  float64 
 5   間取り           615609 non-null  category
 6   面積（㎡）         637351 non-null  float64 
 7   建築年           619117 non-null  float64 
 8   建物の構造         623423 non-null  category
 9   用途            591214 non-null  category
 10  今後の利用目的       275091 non-null  category
 11  都市計画          618786 non-null  category
 12  建ぺい率（％）       614848 non-null  float64 
 13  容積率（％）        614848 non-null  float64 
 14  取引時点          637351 non-null  float64 
 15  改装            580431 non-null  category
 16  取引の事情等        18275 non-null   category
 17  取引価格（総額）_log  637351 non-

In [6]:
import lightgbm as lgb
# scikit-learnサイキットラーン
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

# 学習データと検証データに分ける
df_train, df_val =train_test_split(df, test_size=0.2)

# 目的変数と説明変数に分ける
# 今回のコンペでは目的変数 = 取引価格、それ以外は説明変数
# 目的変数
col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)
#axis=1　はカラムの削除
# 説明変数
val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

# lightgbmでは特別に渡す型にする
trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)
# lgbの最小限パラメータ
params = {
    "objective": "regression", # 回帰タスクなので
    "metrics": "mae" # 評価指標：絶対平均誤差
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, callbacks=[lgb.early_stopping(stopping_rounds=300)])


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11634
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 17
[LightGBM] [Info] Start training from score 7.217416
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 0.0779847


In [9]:
# 確認
vals = model.predict(val_x)
mae(vals, val_y)

0.07798473801595494

In [10]:
# data/test.csvをインポートする
df_test = pd.read_csv("data/test.csv", index_col=0)
df_test = data_pre(df_test)

In [12]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("data/submit_test.csv")
# submit_test.csvに出力されている。
# これをNishikaに投稿する

# 特徴量重要度を見る

In [15]:
model.feature_importance()
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)
# 決定木の分岐に使われたい回数

,importance
地区名,9095
最寄駅：名称,6708
建築年,3523
面積（㎡）,2260
最寄駅：距離（分）,2157
取引時点,1695
間取り,883
市区町村コード,810
都道府県名,583
改装,554


In [ ]:
# 次回、.pyファイルに関数を定義